#### SQLite port_lite database: orders table
#### csv files: orders, ORDERS-PLACED

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 5, 31)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name qty target active spd current change percent reason market xdate'.split()

format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market,xdate
0,20,B,BCT,"3,000",51.00,2,DOS,SET,2022-06-09


In [3]:
orders_log = pd.read_csv('../data/orders-log.csv')
orders_log['trans'] = orders_log.apply(lambda row: categorise(row), axis=1)
orders_log['amount'] = orders_log.qty * orders_log.target
orders_log[orders_log.active == 2].style.background_gradient(cmap = 'Blues')

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
1,B,AP,-4,P75%,SET100,10000,11.000000,11.400000,0.200000,+1.79%,2,2022-05-10,Buy,110000.000000
5,B,BCT,-7,DOS,SET,3000,51.000000,52.750000,-0.750000,-1.40%,2,2022-06-09,Buy,153000.000000
16,B,TTB,-1,1M,SET50,100000,1.260000,1.270000,0.000000,0.00%,2,2022-02-02,Buy,126000.000000


In [4]:
orders_log.head().sort_values('spd',ascending=False).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
4,B,BCPG,-3,3L,SET100,"12,000",11.50,11.80,-0.10,-0.84%,1,2022-02-02,Buy,"138,000.00"
1,B,AP,-4,P75%,SET100,"10,000",11.00,11.40,0.20,+1.79%,2,2022-05-10,Buy,"110,000.00"
0,B,AIT,-10,P25%,SET,"18,000",6.50,7.00,-0.10,-1.41%,0,2022-02-02,Buy,"117,000.00"
3,B,BANPU,-10,P25%3M,SET50,"12,000",11.20,12.20,0.10,+0.83%,0,2022-04-11,Buy,"134,400.00"
2,B,ASK,-20,6L,SET,"3,000",37.75,42.75,-0.50,-1.16%,0,2022-03-03,Buy,"113,250.00"


### End of Tables in the process

### Print to verify before upload file

In [6]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

#df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab = pn.widgets.Tabulator(orders)
df_tab

Tabulator(value=   trade    name  ...)

In [7]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of transactional process

In [8]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
WHERE active = 2
ORDER BY trade, name'''
df_act_2 = pd.read_sql(sql, conlite)
df_act_2.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,B,AP,"10,000",11.00,"110,000.00",P75%,SET100,2,2022-05-10
1,B,BCT,"3,000",51.00,"153,000.00",DOS,SET,2,2022-06-09
2,B,TTB,"100,000",1.26,"126,000.00",1M,SET50,2,2022-02-02


In [9]:
orders_grp = df_act_2.groupby(['trade'])
orders_grp['amount'].sum()

trade
B    389000.0
Name: amount, dtype: float64

In [10]:
mask = df_act_2.trade == 'B'
buy_stocks = df_act_2[mask]
buy_stocks.style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,B,AP,"10,000",11.00,"110,000.00",P75%,SET100,2,2022-05-10
1,B,BCT,"3,000",51.00,"153,000.00",DOS,SET,2,2022-06-09
2,B,TTB,"100,000",1.26,"126,000.00",1M,SET50,2,2022-02-02


In [11]:
cash = 532_000
buy_amt = buy_stocks.amount.sum()
balance = cash - buy_amt 
cash, buy_amt, balance

(532000, 389000.0, 143000.0)

In [12]:
mask = df_act_2.trade == 'S'
sell_stocks = df_act_2[mask]
sell_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate


In [13]:
sell_amt = round(sell_stocks.amount.sum(),2)
balance = cash + sell_amt 
(cash, sell_amt, balance)

(532000, 0.0, 532000.0)

### After call ord-log (must call everytime that orders change)

In [14]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)
df['amount'] = df.qty * df.target

In [15]:
show(df[['trans','name','spd','reason','qty','target','current','active']])

trans,name,spd,reason,qty,target,current,active


In [16]:
mb = (df.trans == 'Buy') & (df.spd >= -7)
df[mb].sort_values(['name'],ascending=[True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
1,B,AP,-4,P75%,SET100,10000,11.00,11.40,0.20,+1.79%,2,2022-05-10,Buy,110000.0
4,B,BCPG,-3,3L,SET100,12000,11.50,11.80,-0.10,-0.84%,1,2022-02-02,Buy,138000.0
5,B,BCT,-7,DOS,SET,3000,51.00,52.75,-0.75,-1.40%,2,2022-06-09,Buy,153000.0
7,B,BPP,-4,3L,SET100,9000,15.60,16.00,-0.10,-0.62%,1,2022-02-02,Buy,140400.0
11,B,ORI,-6,RD05pct,SET100,15000,10.20,10.80,0.10,+0.93%,1,2022-05-09,Buy,153000.0
12,B,SIS,-5,6L,SET,3600,28.75,30.00,-0.25,-0.83%,1,2022-02-02,Buy,103500.0
14,B,SSP,-7,6L,SET,10000,9.50,9.85,-0.05,-0.51%,1,2022-02-02,Buy,95000.0
16,B,TTB,-1,1M,SET50,100000,1.26,1.27,0.00,0.00%,2,2022-02-02,Buy,126000.0


In [17]:
show(df[['trans','name','spd','reason','qty','target','current','percent','active']])

trans,name,spd,reason,qty,target,current,percent,active


In [19]:
ms = (df.trans == 'Sell') & (df.spd <= 3)
df[ms].sort_values(['name'],ascending=[True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
17,S,NER,3,P50%,SET,9000,7.0,6.85,-0.05,-0.72%,1,2022-04-19,Sell,63000.0


In [20]:
df[mb | ms].sort_values(['trade','name'],ascending=[True,True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
1,B,AP,-4,P75%,SET100,10000,11.00,11.40,0.20,+1.79%,2,2022-05-10,Buy,110000.0
4,B,BCPG,-3,3L,SET100,12000,11.50,11.80,-0.10,-0.84%,1,2022-02-02,Buy,138000.0
5,B,BCT,-7,DOS,SET,3000,51.00,52.75,-0.75,-1.40%,2,2022-06-09,Buy,153000.0
7,B,BPP,-4,3L,SET100,9000,15.60,16.00,-0.10,-0.62%,1,2022-02-02,Buy,140400.0
11,B,ORI,-6,RD05pct,SET100,15000,10.20,10.80,0.10,+0.93%,1,2022-05-09,Buy,153000.0
12,B,SIS,-5,6L,SET,3600,28.75,30.00,-0.25,-0.83%,1,2022-02-02,Buy,103500.0
14,B,SSP,-7,6L,SET,10000,9.50,9.85,-0.05,-0.51%,1,2022-02-02,Buy,95000.0
16,B,TTB,-1,1M,SET50,100000,1.26,1.27,0.00,0.00%,2,2022-02-02,Buy,126000.0
17,S,NER,3,P50%,SET,9000,7.00,6.85,-0.05,-0.72%,1,2022-04-19,Sell,63000.0


In [21]:
df[mb | ms].shape[0]

9

In [22]:
mask = (df.trade == 'B')
df[mask].nlargest(5, "amount")[["name","trans","amount","spd"]].style.format(format_dict)

,name,trans,amount,spd
9,GULF,Buy,"276,000.00",-8
10,KKP,Buy,"207,000.00",-10
15,TOP,Buy,"192,600.00",-14
8,GLOBAL,Buy,"177,300.00",-13
13,SPRC,Buy,"162,000.00",-12


### Select source of orders between these two

In [23]:
df_out = df[mb | ms][colt]
df_out

,trans,name,qty,target,active,spd,current,change,percent,reason,market,xdate
1,Buy,AP,10000,11.00,2,-4,11.40,0.20,+1.79%,P75%,SET100,2022-05-10
4,Buy,BCPG,12000,11.50,1,-3,11.80,-0.10,-0.84%,3L,SET100,2022-02-02
5,Buy,BCT,3000,51.00,2,-7,52.75,-0.75,-1.40%,DOS,SET,2022-06-09
7,Buy,BPP,9000,15.60,1,-4,16.00,-0.10,-0.62%,3L,SET100,2022-02-02
11,Buy,ORI,15000,10.20,1,-6,10.80,0.10,+0.93%,RD05pct,SET100,2022-05-09
12,Buy,SIS,3600,28.75,1,-5,30.00,-0.25,-0.83%,6L,SET,2022-02-02
14,Buy,SSP,10000,9.50,1,-7,9.85,-0.05,-0.51%,6L,SET,2022-02-02
16,Buy,TTB,100000,1.26,2,-1,1.27,0.00,0.00%,1M,SET50,2022-02-02
17,Sell,NER,9000,7.00,1,3,6.85,-0.05,-0.72%,P50%,SET,2022-04-19


In [24]:
df_out = df[df.active == 2][colt]
df_out

,trans,name,qty,target,active,spd,current,change,percent,reason,market,xdate
1,Buy,AP,10000,11.00,2,-4,11.40,0.20,+1.79%,P75%,SET100,2022-05-10
5,Buy,BCT,3000,51.00,2,-7,52.75,-0.75,-1.40%,DOS,SET,2022-06-09
16,Buy,TTB,100000,1.26,2,-1,1.27,0.00,0.00%,1M,SET50,2022-02-02


In [25]:
file_name = 'ORDERS-PLACED.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### End of Output ORDERS-PLACED

In [26]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
WHERE ACTIVE = 2
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,B,AP,"10,000",11.00,"110,000.00",P75%,SET100,2,2022-05-10
1,B,BCT,"3,000",51.00,"153,000.00",DOS,SET,2,2022-06-09
2,B,TTB,"100,000",1.26,"126,000.00",1M,SET50,2,2022-02-02


### Evening Process

In [ ]:
new_active = 1
old_active = 2

In [ ]:
sql = """
UPDATE orders 
SET active = %s
WHERE active = %s
"""
sql = sql % (new_active, old_active)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
names = df_out['name']
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

In [ ]:
sql = """
UPDATE orders 
SET active = 2
WHERE name IN (%s)
"""
sql = sql % (in_p)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
#To force orders.csv and orders table to be same copy
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of Evening Process